In [1]:
import csv
import cv2
import time
import os
import numpy as np
from dodo_detector.detection import KeypointObjectDetector, SingleShotDetector
from os.path import isfile, join
from tqdm import tqdm

ImportError: No module named 'dodo_detector.detection'

In [ ]:
test_images_dir = 'test_images'
bases_sift = [
    join('bases_sift/2018', base) for base in os.listdir('bases_sift/2018')
    if base != "originais"
]

log_file = 'logfile.csv'
photos = sorted([
    join(test_images_dir, f) for f in os.listdir(test_images_dir)
    if isfile(join(test_images_dir, f))
])

object_classes = [
    'medicine', 'chocolate_milk', 'heineken', 'yellow_juice', 'red_juice',
    'purple_juice', 'milk_bottle', 'milk_box', 'cereal', 'iron_man', 'shampoo',
    'monster', 'tea_box'
]

tf_frozen_graph = "arquivos_tf/frozen_graph.pb"
tf_label_map = "arquivos_tf/label_map.pbtxt"

In [ ]:
if os.path.exists(log_file):
    os.remove(log_file)

In [ ]:
def detect_and_log(photo,
                   detector,
                   detector_name,
                   log_file,
                   confidence=None,
                   pontos=None,
                   base=None):
    
    # Criando o CSV, se ele não existir
    if not os.path.exists(log_file):
        with open(log_file, 'a') as csvfile:
            objectwriter = csv.writer(
                csvfile,
                delimiter=',',
                quotechar='"',
                quoting=csv.QUOTE_MINIMAL)
            objectwriter.writerow([
                'sensor', 'detector', 'image', 'time (s)', 'confidence',
                'pontos', 'base'
            ] + object_classes)

    img = cv2.imread(photo)
    height, width, channels = img.shape

    if width == 1920:
        sensor = 'camera_hd'
    else:
        sensor = 'kinect_velho'

    start = time.time()
    f, d = detector.from_image(img)  # detecção executada de fato aqui
    end = time.time()
    frame_rate = end - start
    with open(log_file, 'a') as csvfile:
        objectwriter = csv.writer(
            csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        list_csv = [
            sensor, detector_name,
            os.path.splitext(os.path.basename(photo))[0], frame_rate,
            confidence, pontos, base
        ] + [0] * 13

        if len(d) != 0:
            for item in d:
                if item in object_classes:
                    index = object_classes.index(item)
                    list_csv[index + 7] = 1

        objectwriter.writerow(list_csv)

In [ ]:
ssd_detector = SingleShotDetector(tf_frozen_graph, tf_label_map)
for confidence in tqdm(np.arange(0.5, 1.0, 0.05), desc='Confidence'):
    ssd_detector.confidence = confidence
    for photo in tqdm(photos, desc='Image'):
        detect_and_log(photo, ssd_detector, 'ssd', log_file, confidence)

for metodo in ['SIFT', 'RootSIFT']:
    for base in tqdm(bases_sift, desc='Database'):
        sift_detector = KeypointObjectDetector(base, metodo, 'BF', 1)
        for pontos in tqdm(range(10, 51, 10), desc='Min pts'):
            sift_detector.min_points = pontos
            for photo in tqdm(photos, desc='Image'):
                detect_and_log(photo, sift_detector, metodo, log_file, None,
                               pontos, base)